In [21]:
import pandas as pd
import numpy as np
import itertools
from collections import defaultdict
import feather
import gc

#from sklearn.cross_validation import KFold
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import GradientBoostingRegressor
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
cats = ['Green','Fairway','Intermediate Rough','Primary Rough','Fringe','Bunker','Other']

In [4]:
def make_natural_spline_matrix(x,df):
    t = np.percentile(x,np.linspace(0,100,df+2)[1:-1])
    X = x
    def d(x,e_k,e_K):
        def pos(x,e):
            x_ = np.copy(x)
            x_[x<=e] = 0.0
            x_[x>e] = (x_[x>e]-e)**3
            return x_
        return (pos(np.copy(x),e_k) - pos(np.copy(x),e_K))/(e_K-e_k)
    for k in range(1,len(t)-1):
        X = np.column_stack((X,d(np.copy(x),t[k],t[-1]) - d(np.copy(x),t[k-1],t[-1])))
    return X

In [5]:
def tensor_product(mats):
    if len(mats)<2:
        return "I need at least two mats!"
    first = True
    while mats:
        if first:
            res = np.hstack(mats[0].T[:,:,None] * mats[1][None,:,:])
            mats = mats[2:]
            first = False
        else:
            res = np.hstack(res.T[:,:,None] * mats[0][None,:,:])
            mats = mats[1:]
    return res

In [6]:
# data = feather.read_dataframe('./../difficulty.feather')
# data = data[data.Cat=='Primary Rough']
# data.loc[data.Green_to_work_with.isnull(),'Green_to_work_with'] = \
#         data.Green_to_work_with[data.Green_to_work_with.notnull()].mean()
# samp = np.random.choice(range(len(data)),120000 if len(data)>120000 else len(data),replace=False)
# data = data.iloc[samp,:]

# y = data.Shots_taken_from_location.values
# X_dist = make_natural_spline_matrix(data.Distance_from_hole,15)
# X_elev = make_natural_spline_matrix(data.Started_at_Z,8)
# X_gtww = make_natural_spline_matrix(data.Green_to_work_with,6)

In [7]:
dfs = [10,12,14,16],[5,7,9],[4,6,8]

In [8]:
data = feather.read_dataframe('./../difficulty.feather')
data = data[data.Cat=='Primary Rough']
data.loc[data.Green_to_work_with.isnull(),'Green_to_work_with'] = \
        data.Green_to_work_with[data.Green_to_work_with.notnull()].mean()
np.random.seed(45)
train_inds = np.random.choice(range(len(data)),len(data)/2,replace=False)
train_set = set(train_inds.tolist())
test_inds = np.array([ind for ind in range(len(data)) if ind not in train_set])
train = data.iloc[train_inds,:]
test = data.iloc[test_inds,:]
data = None
gc.collect()

18

In [9]:
len(train),len(test)

(551917, 551918)

In [10]:
test.head()

,Shots_taken_from_location,Cat,Distance_from_hole,Green_to_work_with,Started_at_Z
39,3,Primary Rough,386.030789,59.990585,-18.724
62,3,Primary Rough,455.152504,34.871988,-5.515
88,3,Primary Rough,514.180523,13.823223,-3.495
105,2,Primary Rough,436.300429,11.684876,-3.335
119,3,Primary Rough,656.847948,11.084463,-3.530


In [11]:
y_train = train.Shots_taken_from_location.values
X_dist = make_natural_spline_matrix(train.Distance_from_hole.values,13)
X_elev = make_natural_spline_matrix(train.Started_at_Z.values,9)
X_gtww = make_natural_spline_matrix(train.Green_to_work_with.values,8)

X_train = np.hstack((X_dist,X_elev,X_gtww,tensor_product([X_dist,X_elev,X_gtww])))

In [12]:
X_train.shape

(551917, 699)

In [ ]:
lcv = LassoCV(eps=0.001, n_alphas=5, alphas=None, fit_intercept=True, 
              normalize=True, precompute='auto', max_iter=1000, tol=0.0001, 
              copy_X=True, cv=None, verbose=False, n_jobs=1, positive=False, 
              random_state=None, selection='cyclic')

In [ ]:
lcv.fit(X_train,y_train)

In [15]:
hbr = HuberRegressor(epsilon=1.35, max_iter=1000, alpha=0.0001, warm_start=False, 
                     fit_intercept=True, tol=1e-05)

In [23]:
scaler = StandardScaler()

In [ ]:
hbr.fit(scaler.fit_transform(X_train),y_train)

In [ ]:
print np.mean((hbr.predict(X_test) - y_test)**2)

In [19]:
#print np.mean((ols.predict(X_test) - y_test)**2)
#print np.mean((lcv.predict(X_test) - y_test)**2)
print np.mean((hbr.predict(X_test) - y_test)**2)

4.36060621515


In [20]:
hbr.coef_

array([  9.46277885e-12,  -1.24660856e-10,  -1.39964052e-10,
        -1.85833611e-10,  -3.37196340e-10,  -8.94531476e-10,
        -2.99910621e-10,  -1.07279627e-10,  -6.48255277e-11,
        -4.85458523e-11,  -4.13512904e-11,  -4.30373057e-11,
        -1.25089373e-13,  -1.71159634e-12,  -3.21573396e-13,
        -1.19774906e-13,  -5.96019837e-14,  -3.07098973e-14,
        -1.59499755e-14,  -1.12504237e-14,   1.13713023e-12,
        -3.35199912e-12,  -3.10272375e-12,  -3.50736602e-12,
        -3.47718284e-12,  -2.58415386e-12,  -1.79168374e-12,
        -8.45102903e-10,   1.41936674e-09,   1.19433659e-09,
         1.23049946e-09,   1.10546106e-09,   7.50270755e-10,
         4.66435661e-10,  -3.68830576e-09,   8.17727865e-09,
         7.38025448e-09,   8.05745114e-09,   7.65254964e-09,
         5.30127951e-09,   3.45836243e-09,  -3.96802419e-10,
         8.87591548e-10,   8.38258677e-10,   9.48092546e-10,
         9.51960991e-10,   6.87645311e-10,   4.43943713e-10,
        -6.46955050e-11,

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(lcv.alphas_,lcv.mse_path_.mean(1),lw=5,c='purple',label='Mean of 50 Folds');
plt.axhline(np.var(y_train),label='MSE of Prediction with Mean',lw=2,ls='--',c='r');

In [ ]:
(lcv.coef_==0).sum()

In [18]:
y_test = test.Shots_taken_from_location.values
X_dist = make_natural_spline_matrix(test.Distance_from_hole.values,13)
X_elev = make_natural_spline_matrix(test.Started_at_Z.values,9)
X_gtww = make_natural_spline_matrix(test.Green_to_work_with.values,8)

X_test = np.hstack((X_dist,X_elev,X_gtww,tensor_product([X_dist,X_elev,X_gtww])))

In [ ]:
np.mean((ols.predict(X_test) - y_test)**2)

In [ ]:
model = LassoCV(eps=0.001, n_alphas=12, alphas=None, fit_intercept=True, 
              normalize=True, precompute='auto', max_iter=10000, tol=0.0001, 
              copy_X=True, cv=None, verbose=False, n_jobs=1, positive=False, 
              random_state=None, selection='cyclic')
results = defaultdict(list)
for df_1,df_2,df_3 in itertools.product(*dfs):
    
    ## training
    
    y_train = train.Shots_taken_from_location.values
    X_dist = make_natural_spline_matrix(train.Distance_from_hole.values,df_1)
    X_elev = make_natural_spline_matrix(train.Started_at_Z.values,df_2)
    X_gtww = make_natural_spline_matrix(train.Green_to_work_with.values,df_3)
    
    X_train = np.hstack((X_dist,X_elev,X_gtww,tensor_product([X_dist,X_elev,X_gtww])))
    
    model.fit(X_train,y_train)
    
    ## testing

    y_test = test.Shots_taken_from_location.values
    X_dist = make_natural_spline_matrix(test.Distance_from_hole.values,df_1)
    X_elev = make_natural_spline_matrix(test.Started_at_Z.values,df_2)
    X_gtww = make_natural_spline_matrix(test.Green_to_work_with.values,df_3)
    
    X_test = np.hstack((X_dist,X_elev,X_gtww,tensor_product([X_dist,X_elev,X_gtww])))

    score = np.mean((model.predict(X_test) - y_test)**2)
    
    ## print and record results
    print (df_1,df_2,df_3), score
    results['df_dist'].append(df_1)
    results['df_elev'].append(df_2)
    results['df_gtww'].append(df_3)
    results['score'].append(score)

In [ ]:
(((data.Cat=="Green").sum()*0.15510111 +
 (data.Cat=="Fairway").sum()*0.33242987 +
 (data.Cat=="Intermediate Rough").sum()*0.30887963 +
 (data.Cat=="Primary Rough").sum()*0.36587546 +
 (data.Cat=="Fringe").sum()*0.18077784 + 
 (data.Cat=="Bunker").sum()*0.38028543 +
 (data.Cat=="Other").sum()*0.51596645)/len(data))**.5

In [ ]:
gbr = GradientBoostingRegressor(loss='ls', learning_rate=0.01, 
                                n_estimators=1000, subsample=.5, 
                                min_weight_fraction_leaf=0.0, max_depth=3)

In [ ]:
def run():
    errors = []
    for u,cat in enumerate(cats[::-1]):
        data = None
        gc.collect()
        data = feather.read_dataframe('./../difficulty.feather')
        data = data[data.Cat==cat]
        data.loc[data.Green_to_work_with.isnull(),'Green_to_work_with'] = \
        data.Green_to_work_with[data.Green_to_work_with.notnull()].mean()
        
        y = data.values[:,0]
        if cat=='Green': 
            X = data.values[:,np.array([2,4])]
            data = None
            gc.collect()
        else:
            X = data.values[:,2:]
            data = None
            gc.collect()
        gbr.fit(X,y)
        print cat
        print gbr.feature_importances_
        error = np.mean((gbr.predict(X)-y)**2)
        print error
        errors.append(error)
    return errors

In [ ]:
run()

In [ ]:
run()

In [ ]:
(((data.Cat=="Green").sum()*0.11545806607753986 +
 (data.Cat=="Fairway").sum()*0.31740937289 +
 (data.Cat=="Intermediate Rough").sum()*0.300202209856 +
 (data.Cat=="Primary Rough").sum()*0.361347962243 +
 (data.Cat=="Fringe").sum()*0.182508595153 + 
 (data.Cat=="Bunker").sum()*0.372257282426 +
 (data.Cat=="Other").sum()*0.493646743899)/len(data))**.5

In [ ]:
data = data[data.Cat=="Green"]

In [ ]:
X = data.values[:,2:].astype(float)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
pca = PCA()

In [ ]:
X_s = scaler.fit_transform(X)
X_t = pca.fit_transform(X_s)

In [ ]:
X_t.shape

In [ ]:
pca.explained_variance_ratio_

In [ ]:
np.dot(X_t,pca.components_)[0]

In [ ]:
pca.components_

In [ ]:
plt.scatter(X_t[:,0],X_t[:,1],alpha=.03)

In [ ]:
y = data.values[:,0]

In [ ]:
X = data.values[:,np.array([2,4])]
data = None
gc.collect()

In [ ]:
gbr.fit(X,y)

In [ ]:
from sklearn.ensemble.partial_dependence import plot_partial_dependence

In [ ]:
fig,axs = plot_partial_dependence(gbr,X,[1],feature_names='distance',n_jobs=1,grid_resolution=50)